<a href="https://colab.research.google.com/github/syedmahmoodiagents/Stategraph/blob/main/Supervisor_StateGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain langchain-core langchain-community langchain-openai langgraph --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

In [ ]:
!pip install -U langgraph-supervisor --q

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

In [ ]:
import os, getpass

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass()

··········


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
worker_math = create_react_agent(
    llm,
    tools=[],   # Math agent does reasoning only
    name="math_worker"
)

worker_writer = create_react_agent(
    llm,
    tools=[],
    name="writer_worker"
)

/tmp/ipython-input-1485497631.py:1: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  worker_math = create_react_agent(
/tmp/ipython-input-1485497631.py:7: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  worker_writer = create_react_agent(


In [ ]:
supervisor_auto = create_supervisor(
    model=llm,
    agents=[worker_math, worker_writer],
    system_message=(
        "You are a supervisor. "
        "If the question is math -> send to math_worker. "
        "If writing -> send to writer_worker."
    ),
)

In [ ]:
app = supervisor_auto.compile()

In [ ]:
result_auto = app.invoke({
    "messages": [HumanMessage(content="Write a 2-sentence story about a robot.")]
})

In [ ]:
print(result_auto["messages"][-1].content)

In a world where emotions were reserved for humans, a curious little robot named Chip accidentally discovered laughter while trying to understand a joke his creator told. From that day on, he embarked on a mission to bring joy to everyone he met, proving that even circuits could spark happiness.


# Using the Manual StateGraph

In [ ]:
from typing import List, TypedDict
from langgraph.graph import StateGraph, END, START
# from langgraph.constants import START

In [ ]:
class AgentState(TypedDict):
    messages: List

In [ ]:
def worker_math_node(state: AgentState):
    response = llm.invoke([
        {"role": "system", "content": "You are a math specialist."},
        *state["messages"]
    ])
    return {"messages": state["messages"] + [response]}


In [ ]:
def worker_writer_node(state: AgentState):
    response = llm.invoke([
        {"role": "system", "content": "You are a writing specialist."},
        *state["messages"]
    ])
    return {"messages": state["messages"] + [response]}


In [ ]:
from langchain_core.messages import HumanMessage

def supervisor_router(state: AgentState):

    first_human_message_content = ""
    for message in state["messages"]:
        if isinstance(message, HumanMessage):
            first_human_message_content = message.content.lower()
            break

    if "calculate" in first_human_message_content or any(x in first_human_message_content for x in ["add", "sum", "math"]):
        return "worker_math"
    # Default to writer if no specific math keywords are found in the original human message
    return "worker_writer"

In [ ]:
def supervisor_node(state: AgentState):
    sup_msg = llm.invoke([
        {"role": "system",
         "content": "You are a supervisor. Decide which worker to call."},
        *state["messages"]
    ])
    return {"messages": state["messages"] + [sup_msg]}


In [ ]:
graph = StateGraph(AgentState)

# Add nodes
graph.add_node("supervisor", supervisor_node)
graph.add_node("worker_math", worker_math_node)
graph.add_node("worker_writer", worker_writer_node)

# Edges
graph.add_edge(START, "supervisor")

graph.add_conditional_edges(
    "supervisor",
    supervisor_router,
    {
        "worker_math": "worker_math",
        "worker_writer": "worker_writer"
    }
)

# Workers now go directly to END after completing their task
graph.add_edge("worker_math", END)
graph.add_edge("worker_writer", END)

# graph.add_edge("worker_math", "supervisor")
# graph.add_edge("worker_writer", "supervisor")
# graph.add_edge("supervisor", END) # Supervisor's role is now to route to a worker, not provide a final answer directly.

supervisor_manual = graph.compile()

In [ ]:
result_manual = supervisor_manual.invoke({
    "messages": [HumanMessage(content="Write a 2-sentence story about a robot.")]
})

In [ ]:
print(result_manual["messages"][-1].content)

In a future dominated by technology, a small robot named Pixel stumbled upon a forgotten park, overgrown with weeds and litter. Determined to restore its beauty, Pixel worked tirelessly, transforming the area into a vibrant sanctuary that rekindled the community’s connection with nature.
